#Setup

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

import sys 
import glob
import random
import os
import itertools 

from PIL import Image

In [5]:
#run after execute PathSetup in Google Drive part 
from utils import ReplayBuffer

####Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Path setup

In [4]:
#!git clone https://github.com/JuanPabloMF/dl-pytorch
#!unzip /content/drive/MyDrive/Colab_Notebooks/Pytorch_Course/dl-pytorch/datasets/64x64_SIGNS.zip

course_path = '/content/drive/MyDrive/Colab_Notebooks/Pytorch_Course/dl-pytorch/'

sys.path.append(course_path)

####Download Dataset

In [ ]:
zip_path = '/content/drive/MyDrive/Colab_Notebooks/Pytorch_Course/PyTorch_GAN/archive.zip'
zip_output_path = '/content/drive/MyDrive/Colab_Notebooks/Pytorch_Course/PyTorch_GAN/Summer2Winter_yosemite/'

In [ ]:
import shutil
shutil.unpack_archive(zip_path, zip_output_path)

#ANN

####Dataset

In [6]:
class ImageDataset(Dataset):
  def __init__(self, base_dir, transform=None, split='train'):
    self.transform = transforms.Compose(transform)
                                                  #carpeta {split}/ carpeta A / todas las imagenes con formato        
    self.file_A = glob.glob(os.path.join(base_dir,'{}/A/*.*'.format(split)))
    self.file_B = glob.glob(os.path.join(base_dir,'{}/B/*.*'.format(split)))


  def __len__ (self):
    return max(len(self.files_A), len(self.files_B))    

  def __getitem__(self,idx):
    image_A = self.transform(Image.open(self.filesA[idx])) #cargamos una imagen segun idx
    #como son colecciones no apariadas tomamos la imagen B no alineada con A con un random
    image_B = self.transform(Image.open(self.filesB[random.randint(0,len(self.file_B)-1)]))
    return {'A':image_A, 'B':image_B}




####Models

In [7]:
class ResidualBlock(nn.Module):
  def __init__(self, in_features):
    super(ResidualBlock, self).__init__()

    conv_block = [nn.ReflectionPad2d(1), #mejor padding
                  nn.Conv2d(in_features, in_features, 3),
                  nn.InstanceNorm2d(in_features), #Es BatchNorm para GANs
                  nn.ReLU(True),
                  nn.ReflectionPad2d(1), #mejor para conservar la distribucion de la imagen
                  nn.Conv2d(in_features,in_features,3),
                  nn.InstanceNorm2d(in_features)
                  ]
    self.con_block = nn.Sequential(*conv_block)

  def forward(self, x):
    return self.conv_block(x) + x 

In [31]:
class Generator(nn.Module):
  def __init__(self,input_nc,output_nc,n_residual_blocks=9):
    super(Generator,self).__init__()

    #Bloque Convolucional
    model = [nn.ReflectionPad2d(3),
             nn.Conv2d(input_nc, 64, 7), #input chanels, 64 canales, filtro tamaño 7
             nn.InstanceNorm2d(64),
             nn.ReLU(True)
             ]

    in_features = 64
    out_features = in_features * 2

    #ENCODING
    #añadimos dos capas mas
    for _ in range(2):
                #capa de compreccion o encoding, divide x 2 el tamaño de la imagen de entrada I/2
      model += [nn.Conv2d(in_features,out_features,3,stride=2,padding=1), 
                nn.InstanceNorm2d(out_features),
                nn.ReLU(True)
      ]   


      #inflamos x2 la cantidad de canales
      in_features = out_features #128
      out_features = in_features*2 #256

      #Añadimos transformaciones residuales
      for _ in range(n_residual_blocks):
        model+=[ResidualBlock(in_features)]

      #DECODING
      out_features = in_features //2  #//es una division de ints, / de floats, si usamos / data errores de tipo mas adelante

      for _ in range(2):
                  #capa de deconvolucion multiplica x2 la imagen I·2
        model += [nn.ConvTranspose2d(in_features, out_features,3, stride=2, padding=1, output_padding=1),#deconvolucion
                  nn.InstanceNorm2d(out_features),
                  nn.ReLU(True)
                  ] 

        #antes aumentavamos la cantidad de canales, ahora los hacemos disminuir
        in_features = out_features
        out_features = in_features //2

      #la capa de salida es una convolucion que va a aplanar todo
      model += [nn.ReflectionPad2d(3),
                nn.Conv2d(64, output_nc, 7), #I
                nn.Tanh()
                ]
                
      self.model = nn.Sequential(*model)


  #forward propagation
  def forward(self,x):
    return self.model(x)

In [32]:
class Discriminator(nn.Module):
  #PatchGAN: discrimina estilo o textura
  def __init__(self,input_nc):
    super(Discriminator,self).__init__()

    model = [nn.Conv2d(input_nc, 64,4, stride=2, padding=1 ), #I/2
             nn.LeakyReLU(0.2, inplace=True),
             ]

    model += [ nn.Conv2d(64, 128, 4, stride=2, padding=1), #I/2
              nn.InstanceNorm2d(128),
              nn.LeakyReLU(0.2, inplace=True)
              ]

    model += [ nn.Conv2d(128, 256,4,stride=2, padding=1), #I/2
               nn.InstanceNorm2d(256),
               nn.LeakyReLU(0.2, inplace=True)
               ]

    model += [ nn.Conv2d(256, 512,4,stride=2, padding=1), #I-1
               nn.InstanceNorm2d(512),
               nn.LeakyReLU(0.2, inplace=True)
               ] 

    #Flatten 
    model += [ nn.Conv2d(512, 1, 4, padding=1)] #I-1


    self.model = nn.Sequential(*model)

  def forward(self, x):
    x = self.model(x)
    return F.avg_pool2d(x, x.size()[2:]).view(x.size()[0],-1)  #reducira el tamaño de la imagen a un solo valor la salida 

####Instanciando redes, perdidas , optimizadores y perdidas

In [33]:
epochs = 0
n_epochs = 200
batch_size = 4
lr = 0.002
size = 256
input_nc = 3
output_nc = 3
decay_epoch = 100 

cuda = True
device = torch.device('cuda' if cuda else 'cpu')
n_cpu = 8

base_dir = '/content/drive/MyDrive/Colab_Notebooks/Pytorch_Course/PyTorch_GAN/Summer2Winter_yosemite/'


In [34]:
#funcion que ayudara a inicializar los parametros de la red
def weights_init_normal(m):
  
  if isinstance(m, nn.Conv2d): #si la capa es de tipo convolucional
    torch.nn.init.normal(m.weight.data, 0.0, 0.02)
  elif isinstance(m, nn.BatchNorm2d):
    torch.nn.init.normal(m.weight.data, 1.0, 0.02)
    torch.nn.init.constant(m.bias, 0.0)

In [35]:
#instanciamos las redes tanto generadoras como discriminadoras
netG_A2B = Generator(input_nc, output_nc)
netG_B2A = Generator(input_nc, output_nc)
netD_A = Discriminator(input_nc)
netD_B = Discriminator(input_nc)

#aplicamos los pesos a las redes
netG_A2B.apply(weights_init_normal) 
netG_B2A.apply(weights_init_normal) 
netD_A.apply(weights_init_normal) 
netD_B.apply(weights_init_normal)  

#hacemos que las redes vayan en gpu si cuda es True
if cuda:
  netG_A2B.to(device)
  netG_B2A.to(device)
  netD_A.to(device)
  netD_B.to(device) 

#funciones de perdida
criterion_Gan = torch.nn.MSELoss() #mean squared error
criterion_cycle = torch.nn.L1Loss() #perdida L1, para ver la distancia entre las dos imagenes
criterion_identity = torch.nn.L1Loss #para comparar dos imagenes y ver si son similares

<ipython-input-34-985c39b1e59f>:5: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(m.weight.data, 0.0, 0.02)


  Optimizadores y schedules

In [36]:
from torch.optim import lr_scheduler
#optimizadores
#las dor redes generadoras compartiran el mismo optimizador
optimizer_G = torch.optim.Adam(itertools.chain(netG_A2B.parameters(), netG_B2A.parameters()),
                             lr=lr, betas=(0.5,0.999))
optimizer_D_A = torch.optim.Adam(netD_A.parameters(), lr=lr, betas=(0.5,0.999))
optimizer_D_B = torch.optim.Adam(netD_B.parameters(), lr=lr, betas=(0.5,0.999))

#schedulers , actualizar el learning rate de forma dinamica durante el entrenamiento

class LambdaLR():
  def __init__(self, n_epochs, offset, decay_start_epoch):
    assert ((n_epochs - decay_start_epoch) > 0)
    self.n_epochs = n_epochs
    self.offset = offset
    self.decay_start_epoch = decay_start_epoch

  def step(self, epoch):
    return 1 - max(0, epoch + self.offset - self.decay_start_epoch)/(self.n_epochs- self.decay_start_epoch)


lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda=LambdaLR(n_epochs,epochs,decay_epoch).step) #esta funcion dictara cada cuanto y como se actualizara el lr
lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(optimizer_D_A, lr_lambda=LambdaLR(n_epochs,epochs,decay_epoch).step) #esta funcion dictara cada cuanto y como se actualizara el lr
lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(optimizer_D_B, lr_lambda=LambdaLR(n_epochs,epochs,decay_epoch).step) #esta funcion dictara cada cuanto y como se actualizara el lr



####Entrenar

In [ ]:
#inputs y targets
Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor
target_real = Tensor(batch_size).fill_(1.0)
target_fake = Tensor(batch_size).fill_(0.0)

fake_a_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

#Dataloader
transform = [transforms.Resize(int(size*1.12), Image.BICUBIC),
             transforms.RandomCrop(size),
             transforms.RandomHorizontalFlip(),
             transforms.ToTensor(),
             transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
             ]

dataloader = DataLoader(ImageDataset(base_dir,transform=transform),
                       batch_size=batch_size, shuffle=True, num_workers=n_cpu, drop_last=True)

#perdida de GAN
def Gen_GAN_loss(G,D,real,loss,target_real):
  #creo una imagen fake con red generativa
  fake = G(real)
  #hago una prediccion fake con red discriminativa
  pred_fake = D(fake)
  #la perdida
  L = loss(pred_fake,target_real)
  return L, fake

#el cycle loss aplica las dos redes generativas y las compara con la imagen real
def cycle_loss(G1,G2,real,loss):
  recovered = G2(G1(real)) #recuperamos el ciclo g2 g1 de la iamgen real ?¿
  L = loss(recovered, real)

#si la red modifica una imagen de A no queremos que modifique mucho la imagen 
def identity_loss(G, real, loss):
  same = G(real)
  L = loss(same, real)
  return L

loop de entrenamiento

In [ ]:
for epoch in range(epoch,n_epochs):
  for i, batch in enumerate(dataloader):
    real_A = batch['A'].to(device)
    real_B = batch['B'].to(device)

    #Generativas
    optimizer_G.zero_grad()
    loss_GAN_A2B, fake_B = Gen_GAN_loss(netG_A2B, netD_B, real_A, criterion_GAN, target_real)
    loss_GAN_B2A, fake_A = Gen_GAN_loss(netG_B2A, netD_B, real_B, criterion_GAN, target_real)

    loss_cycle_ABA = cycle_loss(netG_A2B, netG_B2A, real_A, criterion_cycle)
    loss_cycle_BAB = cycle_loss(netG_B2A, netG_A2B, real_B, criterion_cycle)

    loss_identity_A = identity_loss(netG__B2A, real_A, criterion_identity)
    loss_identity_B = identity_loss(netG__A2B, real_B, criterion_identity)

    loss_G = (loss_GAN_A2B + loss_GAN_B2A) + 10.0*(loss_cycle_ABA + loss_cycle_BAB) + 5.0 * (loss_identity_A + loss_identity_B)
    loss_G.backward()

    optimizer_G.step

    #Discriminativas
    optimizer_D_A.zero_grad()
    loss_D_A = Disc_GAN_loss(netD_A, fake_A, real_A, fake_A_buffer, criterion_GAN, target_real, target_fake)
    loss_D_A.backward()
    optimizer_D_A.step()

    optimizer_D_B.zero_grad()
    loss_D_B = Disc_GAN_loss(netD_B, fake_B, real_B, fake_B_buffer, criterion_GAN, target_real, target_fake)
    loss_D_B.backward()
    optimizer_D_B.step()

  lr_scheduler_G.step()
  lr_scheduler_D_A.step()
  lr_scheduler_D_B.step()